# Tutorial 1: Agents

## Functionalities:
- Task-based Agents which will break down tasks into subtasks and solve them in bite-sized portions
- Agents with registered functions as skills

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import *
import os

from dotenv import load_dotenv
load_dotenv()

True

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# 1. Agent Basics
- Create an agent by entering your agent's name and description
- Agents are task-based, so they will help generate subtasks to fulfil your main task

- Agents are made to be non-verbose, so they will just focus only on task instruction (Much more efficient compared to conversational-based agentic frameworks like AutoGen)
- Agent's interactions will be stored into `subtasks_completed` by default, which will serve as a memory buffer for future interactions

- **Inputs for Agent**:
    - **agent_name**: String. Name of agent, hinting at what the agent does
    - **agent_description**: String. Short description of what the agent does
    - **max_subtasks**: Int. Default: 5. The maximum number of subtasks the agent can have
    - **verbose**: Bool. Default: True. Whether to print out agent's intermediate thoughts
    - **llm**: Function. The LLM to be used by the Agent
<br/><br/>

- **Agent Internal Parameters**:
    - **Task**: String. The task the agent has been assigned to - Defaults to "No task assigned"
    - **Subtasks Completed**: Dict. The keys are the subtask names and the values are the result of the respective subtask
    - **Is Task Completed**: Bool. Whether the current Task is completed
<br/><br/>

- **Task Running**
    - **reset()**: Resets the Agent Internal Parameters and Subtasks Completed. You should do this at the start of every new task assigned to the Agent to minimise potential confusion of what has been done for this task versus previous tasks
    - **run(task: str, num_subtasks: int = max_subtasks)**: Performs the task. Do note that agent's state will not be reset, so if you want to reset it, call reset() prior to running this. Runs the task for **num_subtasks** steps. If not specified, we will take the **max_subtasks**.
<br/><br/>

- **Give User Output**
    - **reply_user(query: str = '', stateful: bool = True)**: Using all information from subtasks, give a reply about the `query` to the user. If `query` is not given, then it replies based on the current task the agent is doing. If `stateful` is True, saves this query and reply into `subtasks_completed`
<br/><br/>

     - **answer(query, output_format = {'Answer': 'Concise Answer'})**: Using all information from subtasks, give a reply about the `query` to the user. Answers in the given `output_format`
<br/><br/>

    
- **Check status of Agent**:
    - **status()**: Lists out Agent Name, Agent Description, Available Functions (default function is to use the LLM), Task, Subtasks Completed and Is Task Completed
    
## Example Agent Creation
```python
my_agent = Agent('Helpful assistant', 'You are a generalist agent')
```

## Example Agent Task Running - Split the assigned task into subtasks and execute each of them

```python
output = my_agent.run('Give me 5 words rhyming with cool, and make a 4-sentence poem using them')
```

`Subtask identified: Find 5 words that rhyme with 'cool'`

`Getting LLM to perform the following task: Find 5 words that rhyme with 'cool'`
> pool, rule, fool, tool, school

`Subtask identified: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`Getting LLM to perform the following task: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`
> In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.

`Task completed successfully!`

## Check Agent's Status
```python
my_agent.status()
```

`Agent Name: Helpful assistant`

`Agent Description: You are a generalist agent`

`Available Functions: ['use_llm', 'end_task']`

`Task: Give me 5 words rhyming with cool, and make a 4-sentence poem using them`

`Subtasks Completed:`

`Subtask: Find 5 words that rhyme with 'cool'`

`pool, rule, fool, tool, school`

`Subtask: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.`

`Is Task Completed: True`

## Example Agent Reply to User - Reference the subtasks' output to answer the user's query
```python
output = my_agent.reply_user()
```

`
Here are 5 words that rhyme with "cool": pool, rule, fool, tool, school. Here is a 4-sentence poem using these words: "In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool."
`

## Create Agent

In [5]:
# Create your agent by specifying name and description
my_agent = Agent('Helpful assistant', 'You are a generalist agent', llm = llm)

In [6]:
# Show the agent status - By default agent comes equipped with default function `use_llm` which queries the llm
# end_task is to end the current task if it is completed
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


## Automatic Running of Task

In [7]:
# Do the task by subtasks. This does generation to fulfil task
my_agent.reset()
output = my_agent.run('Give me 5 words rhyming with cool, and then make a 4-sentence poem using them')

Observation: No subtasks have been completed yet for the assigned task of generating rhyming words and a poem.

Thoughts: To complete the remainder of the assigned task, I need to first find five words that rhyme with "cool" and then create a four-sentence poem using those words.

Subtask identified: Identify five words that rhyme with "cool" and prepare to use them in a poem.

Getting LLM to perform the following task: Identify five words that rhyme with "cool" and prepare to use them in a poem.

> I have identified five words that rhyme with "cool" which can be effectively used in a poem. The words are:
- school
- pool
- rule
- tool
- drool

These words can be incorporated into various poetic structures, allowing for creative expression. For example, one could write about the experiences in a school, the tranquility of a pool, the importance of a rule, the utility of a tool, or the humorous imagery of drool. Each word offers a unique angle for exploration in poetry, enhancing the ove

In [8]:
# visualise the subtask outputs of the task
print(output)

['I have identified five words that rhyme with "cool" which can be effectively used in a poem. The words are:\n- school\n- pool\n- rule\n- tool\n- drool\n\nThese words can be incorporated into various poetic structures, allowing for creative expression. For example, one could write about the experiences in a school, the tranquility of a pool, the importance of a rule, the utility of a tool, or the humorous imagery of drool. Each word offers a unique angle for exploration in poetry, enhancing the overall theme and rhythm.\n', "In the bright halls of school, where knowledge is the tool,  \nWe gather by the shimmering pool, following every rule.  \nLaughter echoes, and sometimes there's drool,  \nAs we learn and play, life feels wonderfully cool.  \n"]


In [9]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Identify five words that rhyme with "cool" and prepare to use them in a poem.

I have identified five words that rhyme with "cool" which can be effectively used in a poem. The words are:
- school
- pool
- rule
- tool
- drool

These words can be incorporated into various poetic structures, allowing for creative expression. For example, one could write about the experiences in a school, the tranquility of a pool, the importance of a rule, the utility of a tool, or the humorous imagery of drool. Each word offers a unique angle for exploration in poetry, enhancing the overall theme and rhythm.


Subtask: Write a 4-sentence poem using the words school, pool, rule, tool, and drool.

In the bright halls of school, where knowledge is th

In [10]:
# Generates a meaningful reply to the user about the task according to its current state. Functions like a QA bot
# The reply will go into subtasks_completed to store the conversation with the user for future context
output = my_agent.reply_user()

In [11]:
# You can also ask your questions to the agent, and the agent will reply according to its current state. Functions like a QA bot
# Here, we set stateful is false because we do not want this to go into subtasks_completed
output = my_agent.reply_user('Where is the pool?', stateful = False)

In [12]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Identify five words that rhyme with "cool" and prepare to use them in a poem.

I have identified five words that rhyme with "cool" which can be effectively used in a poem. The words are:
- school
- pool
- rule
- tool
- drool

These words can be incorporated into various poetic structures, allowing for creative expression. For example, one could write about the experiences in a school, the tranquility of a pool, the importance of a rule, the utility of a tool, or the humorous imagery of drool. Each word offers a unique angle for exploration in poetry, enhancing the overall theme and rhythm.


Subtask: Write a 4-sentence poem using the words school, pool, rule, tool, and drool.

In the bright halls of school, where knowledge is th

In [13]:
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task. Try to use it in a modular manner and perform one part of the task at a time.
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}



## Running of Task Step by Step
- use `num_subtasks` = 1 to go through step by step when using the .run() method

In [14]:
# Create your agent by specifying name and description
my_agent = Agent('Number Expert', 'You are great with numbers', llm = llm)

In [15]:
# Runs a task for 1 step by editing num_subtasks variable to 1
output = my_agent.run('List me three random numbers from 1 to 50, give me their sum, then generate a math question based on it', num_subtasks = 1)

Observation: No subtasks have been completed yet. The task requires generating three random numbers from 1 to 50, calculating their sum, and creating a math question based on that sum.

Thoughts: To complete the remainder of the Assigned Task, I need to generate three random numbers, calculate their sum, and then formulate a math question using that sum.

Subtask identified: Generate three random numbers from 1 to 50 and calculate their sum.

Getting LLM to perform the following task: Generate three random numbers from 1 to 50 and calculate their sum.

> I generated three random numbers from 1 to 50. The numbers selected are 23, 7, and 34. 
To calculate their sum, I added these numbers together: 
23 + 7 + 34 = 64. 
Therefore, the sum of the three random numbers is 64.




In [16]:
# Visualise the first subtask
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, give me their sum, then generate a math question based on it
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum.

I generated three random numbers from 1 to 50. The numbers selected are 23, 7, and 34. 
To calculate their sum, I added these numbers together: 
23 + 7 + 34 = 64. 
Therefore, the sum of the three random numbers is 64.


Is Task Completed: False


In [17]:
# Runs the task for another step
output = my_agent.run('List me three random numbers from 1 to 50, give me their sum, then generate a math question based on it', num_subtasks = 1)

Observation: I generated three random numbers from 1 to 50. The numbers selected are 23, 7, and 34. 
To calculate their sum, I added these numbers together: 
23 + 7 + 34 = 64. 
Therefore, the sum of the three random numbers is 64.

Thoughts: Now that I have the sum of the numbers, I need to generate a math question based on the sum of 64. 
This could involve creating a question that asks for a number that, when added to another number, equals 64.

Subtask identified: Generate a math question based on the sum of 64.

Getting LLM to perform the following task: Generate a math question based on the sum of 64.

> Based on the sum of 64, I created a math question that challenges the understanding of addition and subtraction. The question is: 
"If you have 64 apples and you give away 16 apples to your friend, how many apples do you have left?" 
This question not only reinforces the concept of subtraction but also encourages the solver to think about real-life applications of math.




In [18]:
# Visualise the first and second subtasks
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, give me their sum, then generate a math question based on it
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum.

I generated three random numbers from 1 to 50. The numbers selected are 23, 7, and 34. 
To calculate their sum, I added these numbers together: 
23 + 7 + 34 = 64. 
Therefore, the sum of the three random numbers is 64.


Subtask: Generate a math question based on the sum of 64.

Based on the sum of 64, I created a math question that challenges the understanding of addition and subtraction. The question is: 
"If you have 64 apples and you give away 16 apples to your friend, how many apples do you have left?" 
This question not only reinforces the concept of subtraction but also encourages the solver to think about real-life applications of m

In [19]:
# see if we need to do another step
output = my_agent.run('List me three random numbers from 1 to 50, give me their sum, then generate a math question based on it', num_subtasks = 1)

Observation: I have generated three random numbers from 1 to 50: 23, 7, and 34. Their sum is 64. I also created a math question based on this sum: "If you have 64 apples and you give away 16 apples to your friend, how many apples do you have left?"

Thoughts: To complete the remainder of the Assigned Task, I need to ensure that the math question is clear and possibly provide an answer to it. Additionally, I can summarize the task completion for the user.

Subtask identified: End Task
Task completed successfully!



In [20]:
# Generates a meaningful reply to the user
my_agent.reply_user()

'I generated three random numbers from 1 to 50. The numbers selected are 23, 7, and 34. \nTo calculate their sum, I added these numbers together: \n23 + 7 + 34 = 64. \nTherefore, the sum of the three random numbers is 64.\n\nBased on the sum of 64, I created a math question that challenges the understanding of addition and subtraction. The question is: \n"If you have 64 apples and you give away 16 apples to your friend, how many apples do you have left?" \nThis question not only reinforces the concept of subtraction but also encourages the solver to think about real-life applications of math.\n'

# 2. Power Up your Agents - Bring in Functions (aka Tools)
- First define the functions, either using class `Function`, or just any Python function with input and output types defined in the signature and with a docstring
- After creating your agent, use `assign_functions` to assign a list of functions of class `Function`, or general Python functions (which will be converted to AsyncFunction)
- Function names will be automatically inferred if not specified
- Proceed to run tasks by using `run()`

```python
# This is an example of an LLM-based function
sentence_style = Function(fn_description = 'Output a sentence with <number> and <entity> in the style of <emotion>', 
                         output_format = {'output': 'sentence'},
                         fn_name = 'sentence_with_number_entities_emotion',
                         llm = llm)

# This is an example of an external user-defined function
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your agent and assign the functions
my_agent = Agent('Helpful assistant', 'You are a generalist agent', 
        llm = llm).assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')
```

`Subtask identified: Convert the binary number 1001 to decimal`
`Calling function binary_to_decimal with parameters {'x': '1001'}`

> {'output1': 9}

`Subtask identified: Generate a happy sentence with the decimal number and a ball`
`Calling function sentence_with_number_entities_emotion with parameters {'number': '9', 'entity': 'ball', 'emotion': 'happy'}`

> {'output': 'I am so happy with my 9 balls.'}

`Task completed successfully!`

- Approach 1: Automatically Run your agent using `run()`

- Approach 2: Manually select and use functions for your task
    - **select_function(task: str)**: Based on the task, output the next function name and input parameters
    - **use_function(function_name: str, function_params: dict, subtask: str = '', stateful: bool = True)**: Uses the function named `function_name` with `function_params`. `stateful` controls whether the output of this function will be saved to `subtasks_completed` under the key of `subtask`
<br/><br/>

- **Assign/Remove Functions**:
    - **assign_functions(function_list: list)**: Assigns a list of functions to the agent
    - **remove_function(function_name: str)**: Removes function named function_name from the list of assigned functions
<br/><br/>

- **Show Functions**:
    - **list_functions()**: Returns the list of functions of the agent
    - **print_functions()**: Prints the list of functions of the agent
<br/><br/>

In [21]:
# Example Internal Function
sentence_style = Function(fn_description = 'Output a sentence with <number> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, fn_name = 'sentence_with_number_entities_emotion',
                     llm = llm)

# Example External Function
def binary_to_decimal(binary_string: str) -> int:
    '''Converts binary_string to integer of base 10'''
    return int(str(binary_string), 2)

# Initialise your agent and assign the functions
my_agent = Agent('Helpful assistant', 'You are a generalist agent', 
        llm = llm).assign_functions([sentence_style, binary_to_decimal])

In [22]:
# Show the functions the agent has
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task. Try to use it in a modular manner and perform one part of the task at a time.
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: sentence_with_number_entities_emotion
Description: Output a sentence with <number> and <entity> in the style of <emotion>
Input: ['number', 'entity', 'emotion']
Output: {'output': 'sentence'}

Name: binary_to_decimal
Description: Converts <binary_string: str> to integer of base 10
Input: ['binary_string']
Output: {'output_1': 'int'}



### Approach 1: Automatic Running of Task

In [23]:
my_agent.reset()
output = my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')

Observation: The Assigned Task requires converting a binary string (1001) to a decimal number and then generating a happy sentence that includes that number and a ball.

Thoughts: To complete the Assigned Task, I need to first convert the binary string 1001 to its decimal equivalent. Once I have that number, I can use it to create a happy sentence that includes the number and the word "ball."

Subtask identified: Convert the binary string 1001 to its decimal equivalent using the binary_to_decimal function.

Calling function binary_to_decimal with parameters {'binary_string': '1001'}
> {'output_1': 9}

Observation: The binary string "1001" has been successfully converted to the number 9.

Thoughts: To complete the remainder of the Assigned Task, I need to generate a happy sentence that includes the number 9 and a ball.

Subtask identified: Use the equipped function to create a sentence that incorporates the number 9 and a ball in a happy style.

Calling function sentence_with_number_ent

In [24]:
# give a response to user
my_agent.reply_user()

'First, the binary string "1001" was converted to a decimal number, resulting in the value 9. \nThen, a happy sentence was generated using that number and the entity "a ball": \n"I am so happy to see 9 bouncing around with a ball!"\n'

In [25]:
# query according to what you need
my_agent.answer('Output only the sentence',
                        output_format = {"Sentence": "type: str"})

{'Sentence': 'I am so happy to see 9 bouncing around with a ball!'}

In [26]:
# show the agent's status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_number_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
Subtasks Completed:
Subtask: binary_to_decimal(binary_string="1001")
{'output_1': 9}

Subtask: sentence_with_number_entities_emotion(number=9, entity="a ball", emotion="happy")
{'output': 'I am so happy to see 9 bouncing around with a ball!\n'}

Subtask: Replying User Query: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
First, the binary string "1001" was converted to a decimal number, resulting in the value 9. 
Then, a happy sentence was generated using that number and the entity "a ball": 
"I am so happy to see 9 bouncing around with a ball!"


Is Task Completed: True


In [27]:
# show the subtasks completed
my_agent.subtasks_completed

{'binary_to_decimal(binary_string="1001")': {'output_1': 9},
 'sentence_with_number_entities_emotion(number=9, entity="a ball", emotion="happy")': {'output': 'I am so happy to see 9 bouncing around with a ball!\n'},
 'Replying User Query: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball': 'First, the binary string "1001" was converted to a decimal number, resulting in the value 9. \nThen, a happy sentence was generated using that number and the entity "a ball": \n"I am so happy to see 9 bouncing around with a ball!"\n'}

In [28]:
# show the agent's detailed observation and thoughts for each subtask
my_agent.thoughts

[{'Observation': 'The Assigned Task requires converting a binary string (1001) to a decimal number and then generating a happy sentence that includes that number and a ball.\n',
  'Thoughts': 'To complete the Assigned Task, I need to first convert the binary string 1001 to its decimal equivalent. Once I have that number, I can use it to create a happy sentence that includes the number and the word "ball."\n',
  'Current Subtask': 'Convert the binary string 1001 to its decimal equivalent using the binary_to_decimal function.\n',
  'Equipped Function Name': 'binary_to_decimal',
  'Equipped Function Inputs': {'binary_string': '1001'}},
 {'Observation': 'The binary string "1001" has been successfully converted to the number 9.\n',
  'Thoughts': 'To complete the remainder of the Assigned Task, I need to generate a happy sentence that includes the number 9 and a ball.\n',
  'Current Subtask': 'Use the equipped function to create a sentence that incorporates the number 9 and a ball in a happy

### Approach 2: Manual Selection and Running of Functions
- If you want to just see what the agent would choose for a hypothetical task, use `select_function`. This will not update the internal state, and will output function name and function params
- If you want specificity in the process, you can just execute the agent's functions yourself using `use_function`
- `use_function`: Uses an agent's function using name and params. Note that by default there will be updating of subtasks_completed when performing the function. In order not to update subtasks_completed, set `stateful = False`

In [29]:
# this should call generate_sentence_with_number_entities_emotion
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Output a sentence with 3, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The Assigned Task requires generating a sentence that includes the number 3, the entity "dog", and conveys the emotion of being happy.

Thoughts: To complete the Assigned Task, I need to utilize the equipped function that can create a sentence with specified parameters. The function "sentence_with_number_entities_emotion" is suitable for this task as it directly addresses the requirements of the task.

Selected function name: sentence_with_number_entities_emotion
Selected function params: {'number': 3, 'entity': 'dog', 'emotion': 'happy'}
Calling function sentence_with_number_entities_emotion with parameters {'number': 3, 'entity': 'dog', 'emotion': 'happy'}
> {'output': 'I have 3 happy dogs that bring joy to my life every day!\n'}



{'output': 'I have 3 happy dogs that bring joy to my life every day!\n'}

In [30]:
# this should call binary_to_decimal
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'What is the decimal representation of binary number 101?')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The binary number 101 has been identified as the input for conversion to decimal.

Thoughts: To complete the remainder of the Assigned Task, I need to convert the binary number 101 into its decimal representation using the appropriate function.

Selected function name: binary_to_decimal
Selected function params: {'binary_string': '101'}
Calling function binary_to_decimal with parameters {'binary_string': '101'}
> {'output_1': 5}



{'output_1': 5}

In [31]:
# this should call use_llm
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Research on the benefits of exercise based on LLM')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: Research has been conducted on various benefits of exercise, including physical health improvements, mental well-being, and social interaction. 
Subtasks Completed include gathering initial data and identifying key areas of focus such as cardiovascular health, weight management, and mood enhancement.

Thoughts: To complete the remainder of the Assigned Task, I need to synthesize the gathered information into coherent insights. 
This may involve summarizing findings, highlighting statistics, and providing examples of how exercise impacts different aspects of health.

Selected function name: use_llm
Selected function params: {'instruction': 'Summarize the benefits of exercise into key points, focusing on physical, mental, and social benefits, and prepare to present these findings.\n'}
Getting LLM to perform the following task: Summarize the benefits of exercise into key points, focusing on physical, mental, and social benefits, and prepare to present these findings.

> The b

{'Detailed Outcome': 'The benefits of exercise can be summarized into three key categories: physical, mental, and social benefits.\n\nPhysical Benefits:\n - Improves cardiovascular health by strengthening the heart and improving circulation.\n - Aids in weight management by burning calories and increasing metabolism.\n - Enhances muscle strength and endurance, leading to better overall physical performance.\n - Increases flexibility and balance, reducing the risk of injuries.\n - Boosts the immune system, helping to prevent illnesses.\n\nMental Benefits:\n - Reduces symptoms of anxiety and depression by releasing endorphins, which are natural mood lifters.\n - Improves cognitive function and memory through increased blood flow to the brain.\n - Enhances sleep quality, leading to better rest and recovery.\n - Increases self-esteem and confidence as individuals achieve fitness goals.\n - Provides a sense of accomplishment and purpose.\n\nSocial Benefits:\n - Encourages social interaction

# 3. AsyncAgent

- `AsyncAgent` works the same way as `Agent`, only much faster due to parallelisation of tasks
- It can only be assigned functions of class `AsyncFunction`, or general Python functions (which will be converted to AsyncFunction)
- If you define your own `AsyncFunction`, you should define the fn_name as well if it is not an External Function
- As a rule of thumb, just add the `await` keyword to any function that you run with the `AsyncAgent`

#### Example LLM in Async Mode
```python
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
```

#### Example Agentic Workflow
```python
# This is an example of an LLM-based function
sentence_style = AsyncFunction(fn_description = 'Output a sentence with <number> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, 
                     fn_name = 'sentence_with_number_entities_emotion',
                     llm = llm_async)

# This is an example of an external user-defined function (see Tutorial 0)
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your Agent and assign functions
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent', 
                      llm = llm_async).assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = await my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')
```

In [32]:
# Define an Async LLM function
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [33]:
# This is an example of an LLM-based function
sentence_style = AsyncFunction(fn_description = 'Output a sentence with <number> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, 
                     fn_name = 'sentence_with_number_entities_emotion',
                     llm = llm_async)

# This is an example of an external user-defined function (see Tutorial 0)
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your Agent and assign functions
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent', 
                      llm = llm_async).assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = await my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')

Observation: The Assigned Task requires converting a binary string (1001) to a decimal number and then generating a happy sentence that includes that number and the word "ball".

Thoughts: To complete the remainder of the Assigned Task, I need to first use the equipped function to convert the binary string to a decimal number. Once I have that number, I can then use another equipped function to create a happy sentence that includes the number and the word "ball".

Subtask identified: Convert the binary string '1001' to a decimal number using the binary_to_decimal function.

Calling function binary_to_decimal with parameters {'binary_number': '1001'}
> {'output_1': 9}

Observation: The binary string "1001" has been successfully converted to the number 9.

Thoughts: To complete the remainder of the Assigned Task, I need to generate a happy sentence that includes the number 9 and a ball.

Subtask identified: Use the equipped function to create a sentence that incorporates the number 9 and

In [34]:
await my_agent.reply_user()

'To complete the assigned task, I first converted the binary string "1001" to its decimal equivalent. The conversion process resulted in the number 9. This was achieved using the binary_to_decimal function, which confirmed that the output for the binary number "1001" is 9.\n\nNext, I generated a sentence that incorporates the number 9, the entity "a ball," and the emotion "happy." The sentence created is: "I have 9 shiny balls that bring me so much joy!" This sentence effectively conveys the requested elements of the task, expressing happiness associated with the number of balls.\n\nTherefore, the complete response to the assigned task is as follows:\n- The decimal conversion of binary "1001" is 9.\n- The happy sentence generated is: "I have 9 shiny balls that bring me so much joy!"\n'

In [35]:
await my_agent.answer("What is the number?",
                      output_format = {"Binary": "type: str", 
                                       "Integer": "type: int"})

{'Binary': '1001', 'Integer': 9}

In [36]:
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_number_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
Subtasks Completed:
Subtask: binary_to_decimal(binary_number="1001")
{'output_1': 9}

Subtask: sentence_with_number_entities_emotion(number=9, entity="a ball", emotion="happy")
{'output': 'I have 9 shiny balls that bring me so much joy!\n'}

Subtask: Replying User Query: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
To complete the assigned task, I first converted the binary string "1001" to its decimal equivalent. The conversion process resulted in the number 9. This was achieved using the binary_to_decimal function, which confirmed that the output for the binary number "1001" is 9.

Next, I generated a s

In [37]:
# this should call generate_sentence_with_emotion
my_agent.reset()
function_name, function_params = await my_agent.select_function(
    task = 'Output a sentence with bell, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

await my_agent.use_function(function_name, function_params, stateful = False)

Observation: The task requires generating a sentence that includes the words "bell," "dog," and conveys a "happy" emotion.

Thoughts: To complete the task, I need to use the equipped function that can create a sentence with specified words and emotions. The function "sentence_with_number_entities_emotion" is suitable for this purpose.

Selected function name: sentence_with_number_entities_emotion
Selected function params: {'number': 1, 'entity': 'dog', 'emotion': 'happy'}
Calling function sentence_with_number_entities_emotion with parameters {'number': 1, 'entity': 'dog', 'emotion': 'happy'}
> {'output': 'There is 1 happy dog wagging its tail joyfully in the sun.\n'}



{'output': 'There is 1 happy dog wagging its tail joyfully in the sun.\n'}

# Saving and Loading Agents
Sometimes you want to configure your agents and save them and load them elsewhere, while maintaining the current agent state

- When you use the `save_agent` function, we store the entire agent's internal state, include name, description, list of functions, subtasks history and all other internal variables into a pickle file
- When you use the `load_agent` function, and we will load the entire agent saved in the pickle file into the existing agent

Key functions:
- **save_agent(pickle_file_name: str)**: Saves the agent's internal parameters to a pickle file named pickle_file_name (include .pkl), returns the pickle file
- **load_agent(pickle_file_name: str)**: Loads the agent's internal parameters from a pickle file named pickle_file_name (include .pkl), returns loaded agent

#### Example 1: Saving Agent
```python
my_agent.save_agent('myagent.pkl')
```

#### Example Output
```Agent saved to myagent.pkl```

#### Example 2: Loading Agent
```python
new_agent = Agent().load_agent('myagent.pkl')
```

#### Example Output
```Agent loaded from myagent.pkl```

In [38]:
# run one task to add something to subtasks_completed
my_agent.reset()
output = my_agent.run('What is the decimal representation of binary number 101?')

In [39]:
# see status of agent before saving
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_number_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [40]:
# save agent
my_agent.save_agent('myagent.pkl')

Agent saved to myagent.pkl


In [41]:
# load agent using load_agent code
new_agent = Agent().load_agent('myagent.pkl')

Agent loaded from myagent.pkl


In [42]:
# see status of loaded agent (which also includes what is saved in subtasks_completed, and shared_variables)
new_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_number_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False
